# huggingface/trl SFTTrainerを使用したファインチューニング
参考：https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?huggingface%2Ftrl%20SFTTrainer  
※ ディスク領域が必要なので、このNBを実行する場合は、gitクローンする段階で/mnt/tmp（※ 一時領域）などに配置すると良い。

## 準備

### 必要なパッケージをインストール

#### サンドボックス

In [1]:
!pip cache purge
# !pip install --upgrade XXXX

Files removed: 0


#### 実際にインストールしたもの一覧

- datasets : 自然言語処理のサンプルデータセットを簡単に取り扱えるライブラリ
- transformers : Hugging Face の transformers ライブラリ
- accelerate : Hugging Face の accelerate ライブラリは、TPU/GPU/CPUでの実行を同じコードで記述できる
- trl : Hugging Face の trl (Transformer Reinforcement Learning) ライブラリ（SFT用）
- peft : Hugging Face の peft (Parameter-Efficient Fine Tuning) ライブラリ（LoRA用）
```bash
!pip install --upgrade datasets
!pip install --upgrade transformers
!pip install --upgrade accelerate
!pip install --upgrade trl
!pip install --upgrade peft
```

#### 以下のエラーが出るので、インストール後に一旦再起動
```
ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`
```
参考：https://github.com/huggingface/transformers/issues/24147

#### インストール結果の確認

In [2]:
!pip list

Package                   Version
------------------------- --------------
accelerate                1.8.1
aiohappyeyeballs          2.6.1
aiohttp                   3.12.13
aiosignal                 1.4.0
anyio                     4.9.0
argon2-cffi               25.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bleach                    6.2.0
certifi                   2025.7.9
cffi                      1.17.1
charset-normalizer        3.4.2
comm                      0.2.2
datasets                  4.0.0
debugpy                   1.8.14
decorator                 5.2.1
defusedxml                0.7.1
dill                      0.3.8
executing                 2.2.0
fastjsonschema            2.21.1
filelock                  3.13.1
fqdn                      1.5.1
frozenlist                1.7.0
fsspec         

### インストールしたパッケージをインポート

In [3]:
import torch
import datasets
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from trl import DataCollatorForCompletionOnlyLM, SFTTrainer

### 使用する変数を定義する

In [4]:
# DATA
TEST_FILE_PATH = 'finetuning/data/test1.jsonl'

# MODEL
MODEL_URI                        = 'meta-llama/Llama-3.2-1B-Instruct'
MODEL_PATH      =           'finetuning/models/Llama-3.2-1B-Instruct'
FTED_MODEL_PATH = 'finetuning/finetuned_models/Llama-3.2-1B-Instruct'

### モデル・トークナイザ

#### NotebookからHugging Face Model Hubとやり取り

In [5]:
from huggingface_hub import notebook_login
notebook_login()

### モデルとトークナイザのダウンロードとセーブ
初回のみなのでMarkdownにしておく。

```python
# model
model = transformers.AutoModelForCausalLM.from_pretrained(
    MODEL_URI,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    force_download=True
  )
model.save_pretrained(MODEL_PATH) 

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_URI)
tokenizer.save_pretrained(MODEL_PATH) 
```

## ファインチューニング

### データの読み込み

#### 間違った方法
- なお、以下で表示される結果の「Datasetのtrain」と「DatasetDictのtrain」は対応している訳ではない。
- これは、`datasets.DatasetDict({'train': dataset})`のtrain → xxxxx に変えて実行すると解る。

In [6]:
test_file = "./data/test.jsonl"
dataset = datasets.load_dataset("json", data_files=test_file)
dataset = datasets.DatasetDict({'train': dataset})
dataset

DatasetDict({
    train: DatasetDict({
        train: Dataset({
            features: ['question', 'answer'],
            num_rows: 5
        })
    })
})

#### 正しい方法

In [7]:
dataset = datasets.load_dataset("json", data_files=TEST_FILE_PATH, split="train")
dataset = datasets.DatasetDict({'train': dataset})
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 5
    })
})

### SFTTrainerの定義

#### model, tokenizer
モデルとトークナイザのロード

In [8]:
# model
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

#### training_args

In [9]:
training_args = TrainingArguments('test_trainer') # output_dirの指定になる。

# 「OutOfMemoryError: CUDA out of memory.」への対応
# training_args = TrainingArguments(
#     output_dir='test_trainer',
#     per_device_train_batch_size=1, # 8, 4, 2
#     gradient_checkpointing=True,
#     bf16=True, #fp16=True  # NVIDIA Ampere 以降の GPU なら bf16=True も検討
# )

#### formatting_func

In [10]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"Please answer the question.\n\n### question\n{example['question'][i]}\n\n### answer\n{example['answer'][i]}<|endoftext|>"
        output_texts.append(text)
    return output_texts

#### data_collator

In [11]:
response_template = '### answer\n'
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

#### SFTTrainerの定義

In [12]:
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=dataset['train'], 
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

Applying formatting function to train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Applying formatting function to train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

#### SFTTrainer.train_datasetを確認

In [13]:
print(trainer.train_dataset)
# SFTTrainerの出力が途中で止まることがあるので4件目を確認
print(tokenizer.decode(trainer.train_dataset[3]['input_ids']))

Dataset({
    features: ['question', 'answer', 'text', 'input_ids'],
    num_rows: 5
})
<|begin_of_text|>Please answer the question.

### question
クライアント、サーバ、データストア、その間のネットワークに関連した問題が発生します。

### answer
SI界隈でよく発生するインフラストラクチャに関する性能問題にはどのようなものがありますか？<|endoftext|><|eot_id|>


### SFTTrainer.trainで実行

#### OutOfMemoryError: CUDA out of memory. 対策
要るか要らないか解らないので、取り敢えずMarkdownで。

```python
#`use_cache=True`は勾配チェックポイントと互換性が無いので併用できない。
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
```

#### 実際にSFTを実行

In [ ]:
trainer.train()

#### SFTされたモデルとトークナイザのセーブ

In [ ]:
tokenizer.from_pretrained(FTED_MODEL_PATH)
trainer.from_pretrained(FTED_MODEL_PATH)

## SFTされたモデルで推論を実行

### SFTされたモデルとトークナイザのロード

In [ ]:
tokenizer.save_pretrained(FTED_MODEL_PATH) # ,device_map="auto", torch_dtype=torch.float16)
trainer.save_model(FTED_MODEL_PATH)

### プロンプトをトークン化
`pt`は、PyTorch テンソル（torch.Tensor）の意味

In [ ]:
inputs = tokenizer('...プロンプト...', return_tensors='pt').to(model.device)
print(inputs) # テンソルのディクショナリになっている。

### 推論を実行
- 推論時なので勾配計算しない`no_grad`プロックに入れて実行
- `**inputs`として渡すと「ディクショナリのキー・値」が「キーワード引数・引数」に展開される。

In [ ]:
with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

### 推論結果を表示
推論結果をデトークン化して表示。

In [ ]:
output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)